# Jupyter notebook sample

In [ ]:
import asyncio
import json
import os
import random
import re
import sqlite3
import threading
import time
import uuid
from abc import ABC, abstractmethod
from datetime import datetime
from math import floor
from tkinter import Tk, Toplevel, Canvas, Label, Button, Entry, StringVar, PhotoImage
from tkinter import ttk

import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image, ImageTk
from google.auth.transport.requests import Request
from google.oauth2 import id_token
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build



SCOPES = ['https://www.googleapis.com/auth/fitness.activity.read']


In [ ]:
def google_login():
    flow = InstalledAppFlow.from_client_secrets_file(
        "client_secret.json",
        scopes=["openid", "https://www.googleapis.com/auth/userinfo.email"]
    )
    credentials = flow.run_local_server()

    request = Request()
    info = id_token.verify_oauth2_token(credentials.id_token, request)

    email = info.get("email")
    return email


In [ ]:
def connect_db():
    with sqlite3.connect("game.db", timeout=10, check_same_thread=False) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS characters (
                id TEXT,
                name TEXT,
                age INTEGER,
                gender TEXT,
                weight REAL,
                height REAL,
                xp INTEGER,
                level INTEGER,
                coins INTEGER,
                daily_steps INTEGER,
                all_time_steps INTEGER,
                beaten_enemies INTEGER,
                email TEXT PRIMARY KEY,
                class TEXT,
                status INTEGER
            )
        ''')
        conn.commit()


connect_db()


In [1]:
class Character(ABC):
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, name, age, gender, weight, height, email):
        self._name = name
        self._age = age
        self._gender = gender
        self._weight = weight
        self._height = height
        self._id = str(uuid.uuid4())
        self._email = email
        self._status = 0

        self._beaten_enemies = 0
        self._sleep_time = None
        self._hydration = None
        self._health = None
        self._damage = None
        self._heal_amount = None
        self._progress = Progression(self._id)
        self._energy = StepEnergyMeter(self._id)

    def set_status(self, amount):
        self._status = amount

    def get_status(self):
        return self._status

    def get_damage(self):
        self.damage_calculator()
        return self._damage

    def get_health(self):
        self.health_calculator()
        return self._health

    def get_heal_amount(self):
        self.heal_calculator()
        return self._heal_amount

    def get_beaten_enemies(self):
        return self._beaten_enemies

    def set_beaten_enemies(self, amount):
        self._beaten_enemies = amount

    @abstractmethod
    def health_calculator(self):
        pass

    def save_to_db(self):
        print("Saving...")
        retries = 5
        delay = 3

        for i in range(retries):
            try:
                with sqlite3.connect("game.db", timeout=10, check_same_thread=False) as conn:
                    cursor = conn.cursor()
                    cursor.execute('''
                        INSERT OR REPLACE INTO characters
                        (id, name, age, gender, weight, height, xp, level, coins, daily_steps,
                         all_time_steps, beaten_enemies, email, class, status)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        self._id, self._name, self._age, self._gender, self._weight, self._height,
                        self._progress.get_xp(), self._progress.get_level(), self._progress.get_coins(),
                        self._energy.get_daily_steps(), self._energy.get_all_time_steps(),
                        self._beaten_enemies, self._email, type(self).__name__, self.get_status()
                    ))
                    conn.commit()
                break
            except sqlite3.OperationalError as e:
                print(e)
                print(f"DB is locked, retry {i + 1}/{retries}...")
                time.sleep(delay)
        else:
            print("Failed to save to DB after retries.")

    def set_email(self, email):
        self._email = email

    def set_id(self, id):
        self._id = id

    def set_name(self, name):
        self._name = name

    @abstractmethod
    def damage_calculator(self):
        pass

    @abstractmethod
    def heal_calculator(self):
        pass

    def change_name(self, new_name, frame):
        res = Toplevel(frame, bg='#878E99')
        res.title("Result")
        if self._progress.get_coins() >= 500:
            self._name = new_name
            lab = Label(res, text="You successfully changed your hero's name for 500 coins!")
            coins = self._progress.get_coins() - 500
            self._progress.set_coins(coins)
            conn, cursor = connect_db()
            cursor.execute('UPDATE characters SET name = ? WHERE id = ?', (self._name, self._id))
            conn.commit()
            conn.close()
        else:
            lab = Label(res, text="Not enough coins, name change costs 500 coins!")
        lab.pack()
        btn_ex = Button(res, text="OK", command=lambda: destr_windows(res, frame))
        btn_ex.pack()

    def add_weight(self, amount):
        self._weight = amount

    def add_height(self, amount):
        self._height = amount

    def get_name(self):
        return self._name

    def get_hydration(self):
        return self._hydration

    def set_hydration(self, amount):
        self._hydration = amount

    def add_hydration(self, amount):
        if float(amount) > 0:
            self._hydration = min(8, float(amount))

    def sleep_time(self, amount):
        self._sleep_time = amount

    def get_sleep_time(self):
        return self._sleep_time

    def set_sleep_time(self, amount):
        if float(amount) > 0:
            self._sleep_time = min(8, float(amount))

    def get_email(self):
        return self._email




class Warrior(Character):
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, name, age, gender, weight, height, email):
        super().__init__(name, age, gender, weight, height, email)
        self._health = None
        self._damage = None
        self._heal_amount = None

    def health_calculator(self):
        hydration = self._hydration or 0
        sleep_time = self._sleep_time or 0
        self._health = self._progress.get_level() + (50 * (hydration + sleep_time))

    def damage_calculator(self):
        hydration = self._hydration or 0
        sleep_time = self._sleep_time or 0
        self._damage = pow(self._progress.get_level(), (sleep_time / 4 + hydration / 2))

    def heal_calculator(self):
        self._heal_amount = self._progress.get_level() * 5

    def get_damage(self):
        self.damage_calculator()
        return round(self._damage, 0)

    def get_health(self):
        self.health_calculator()
        return self._health

    def get_heal_amount(self):
        self.heal_calculator()
        return self._heal_amount

    def set_sleep_time(self, amount):
        self._sleep_time = min(8, float(amount))

    def set_hydration(self, amount):
        self._hydration = min(5, float(amount))


class Mage(Character):
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    def __init__(self, name, age, gender, weight, height, email):
        super().__init__(name, age, gender, weight, height, email)
        self._health = None
        self._damage = None
        self._heal_amount = None

    def health_calculator(self):
        hydration = self._hydration or 0
        sleep_time = self._sleep_time or 0
        self._health = self._progress.get_level() + (20 * (hydration + sleep_time))

    def damage_calculator(self):
        hydration = self._hydration or 0
        sleep_time = self._sleep_time or 0
        self._damage = pow(self._progress.get_level(), (sleep_time / 2 + hydration))

    def heal_calculator(self):
        self._heal_amount = self._progress.get_level() * 10

    def get_damage(self):
        self.damage_calculator()
        return round(self._damage, 0)

    def get_health(self):
        self.health_calculator()
        return self._health

    def get_heal_amount(self):
        self.heal_calculator()
        return self._heal_amount

    def set_sleep_time(self, amount):
        self._sleep_time = min(8, float(amount))

    def set_hydration(self, amount):
        self._hydration = min(8, float(amount))


NameError: name 'ABC' is not defined

In [ ]:
class Progression:
    def __init__(self, id_):
        self._id = id_
        self._level = 1
        self._xp = 0
        self._coins = 0

    def get_coins(self):
        return self._coins

    def set_coins(self, coins):
        self._coins = coins

    def get_xp(self):
        return round(self._xp, 0)

    def set_xp(self, xp):
        self._xp = xp

    def add_coins(self, amount):
        self._coins += amount

    def set_level(self, level):
        self._level = level

    def get_level(self):
        A, B, C, D, E = 50, 1.5, 200, 100, 5
        for level in range(1, 1000):
            if self._xp <= E * (A * (level ** B) + D * level + C):
                return level
        return 999

    def xp_calculation(self, amount):
        self._xp += amount

    def steps_to_xp(self):
        pass

    def add_xp(self, amount):
        print("Adding XP:", amount)
        self._xp += amount


In [ ]:
class StepEnergyMeter:
    def __init__(self, id_):
        self._id = id_
        self._total_steps = 0
        self._daily_steps = 0
        self._temp = 0

    def set_daily_steps(self, steps):
        self._daily_steps = steps

    def get_daily_steps(self):
        return self._daily_steps

    def set_all_time_steps(self, steps):
        self._total_steps = steps

    def update_all_time_steps(self):
        # self._total_steps += self.daily_steps_from_google()
        pass

    def daily_steps_from_google(self):
        creds = None
        token_name = player.get_email()

        if os.path.exists(f'{token_name}.json'):
            print("Found token, loading credentials...")
            creds = Credentials.from_authorized_user_file(f'{token_name}.json', SCOPES)
        else:
            print("Token not found, starting OAuth flow.")

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                print("Refreshing token...")
                creds.refresh(Request())
            else:
                print("Running OAuth flow...")
                flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)

        with open(f'{token_name}.json', 'w') as token:
            token.write(creds.to_json())

        today = datetime.today().strftime("%Y-%m-%d")
        start_time = int(time.mktime(time.strptime(f"{today} 00:00:00", "%Y-%m-%d %H:%M:%S"))) * 1000
        end_time = int(time.mktime(time.strptime(f"{today} 23:59:59", "%Y-%m-%d %H:%M:%S"))) * 1000

        url = "https://www.googleapis.com/fitness/v1/users/me/dataset:aggregate"
        headers = {
            "Authorization": f"Bearer {creds.token}",
            "Content-Type": "application/json"
        }
        body = {
            "aggregateBy": [{
                "dataTypeName": "derived:com.google.step_count.delta",
                "dataSourceId": "derived:com.google.step_count.delta:com.google.android.gms:estimated_steps"
            }],
            "bucketByTime": {"durationMillis": 86400000},
            "startTimeMillis": start_time,
            "endTimeMillis": end_time
        }

        response = requests.post(url, headers=headers, json=body)
        data = response.json()

        total_steps = 0
        for bucket in data.get("bucket", []):
            for dataset in bucket.get("dataset", []):
                for point in dataset.get("point", []):
                    for value in point.get("value", []):
                        total_steps += value.get("intVal", 0)

        print(f"Total steps today: {total_steps}")
        previous_steps = self.get_daily_steps()

        player._progress.add_xp(-player._energy.get_daily_steps())
        player._progress.add_xp(total_steps)

        if total_steps != previous_steps:
            self._daily_steps = total_steps
            player._progress.steps_to_xp()

        print(f"Step difference: {self._daily_steps - previous_steps}")
        return self._daily_steps - previous_steps

    def get_temp(self):
        return self._temp

    def get_all_time_steps(self):
        return self._total_steps

In [ ]:
class Battle:
    def __init__(self, player, enemy, frame):
        self._frame = frame
        self._player = player
        self._enemy = enemy

        self._player.health_calculator()
        self._player.damage_calculator()
        self._player.heal_calculator()

        self._player_current_health = self._player.get_health()
        self._enemy_current_health = self._enemy.get_health()

        global Lab_act
        Lab_act = Label(self._frame, text="Get ready for your action", wraplength=300)
        Lab_act.place(x=30, y=330)

    def get_current_hp(self):
        return self._player_current_health

    def get_enemy_current_hp(self):
        return self._enemy_current_health

    def take_damage(self):
        damage = round(self._enemy.get_damage(), 0)
        self._player_current_health -= damage

        if self._player_current_health <= 0:
            result = Toplevel(self._frame, bg='#878E99')
            result.title("Result")
            Label(result, text="You Lost!").pack()
            Button(result, text="OK", command=lambda: dest_windows_lose(result, self._frame)).pack()

        Lab_td = Label(
            self._frame,
            text=f"You received {damage} damage! Think about defense"
        )
        Lab_td.place(x=70, y=300)

    def heal(self):
        before_heal = self._player_current_health
        heal_amount = self._player.get_heal_amount()
        self._player_current_health = min(
            self._player_current_health + heal_amount,
            self._player.get_health()
        )

        actual_healed = self._player_current_health - before_heal
        Lab_act.config(
            text=f"You were healed for {actual_healed} health. You can think about doctor occupation"
        )

        self.take_damage()

    def deal_damage(self):
        damage = self._player.get_damage()
        self._enemy_current_health -= damage

        if self._enemy_current_health <= 0:
            result = Toplevel(self._frame, bg='#878E99')
            result.title("Result")
            Label(result, text="You Won!").pack()
            Button(result, text="OK", command=lambda: dest_windows_won(result, self._frame)).pack()

        Lab_act.config(text=f"You dealt {damage} damage!")
        self.take_damage()

In [ ]:
class Enemy:
    def __init__(self, hp, damage, name):
        self._hp = hp
        self._damage = damage
        self._name = name

    def get_health(self):
        return self._hp

    def get_damage(self):
        return self._damage

    def get_name(self):
        return self._name


Enemies = []

with open("Enemies.txt", "r") as file:
    for line in file:
        currentline = line.strip().split(",")
        if len(currentline) == 3:
            hp, damage, name = int(currentline[0]), int(currentline[1]), currentline[2]
            Enemies.append(Enemy(hp, damage, name))

for enemy in Enemies:
    print(enemy.get_name())


 "Tralaleilo Tralala"
 "Goblin Grunt"
 "Cave Rat"
 "Slimey McBlob"
 "Dark Elf Archer"
 "Skeleton Warrior"
 "Flame Imp"
 "Frost Bat"
 "Shadow Beast"
 "Swamp Lurker"
 "Lava Golem"
 "Necromancer"
 "Void Elemental"
 "Iron Minotaur"
 "Crystal Dragonling"
 "Demon Overlord"


In [9]:

global quotes, hydration, dBoss_killed,player

dBoss_killed=False

with open("quotes.txt", "r", encoding="utf-8") as file:
    quotes = [line.strip() for line in file if line.strip()]

today = datetime.today().strftime("%Y-%m-%d")


random.seed(today)

quotes_number = random.randint(0, len(quotes)-1)
print(quotes_number)

root = Tk()
root.title("HeroFit")
root.geometry("350x500")
root.resizable(False, False)
root.configure(bg='#878E99')



# ======= Game Frame =======
frame_game = Frame(root, bg="#878E99")


player = None
steps = None
hero_level = None


def destr_windows(frame1, frame2):
    frame1.destroy()
    frame2.destroy()

def ch_save(name,Chang):
    player.change_name(name,Chang)



def changing():
    Chang = Toplevel(frame_game,bg='#878E99')
    Chang.title("Changing name")

    Ch_lab = Label(Chang, text="Enter name you want to change to")
    Ch_lab.pack(pady=5)
    Ch_ent = Entry(Chang)
    Ch_ent.pack()
    Btn_save = Button(Chang, text="Save", command=lambda: ch_save(Ch_ent.get(),Chang))
    Btn_save.pack()

def dest_windows_lose(Result,Battl):
    Result.destroy()
    Battl.destroy()

def dest_windows_won(Result,Battl):
    player.set_beaten_enemies(player.get_beaten_enemies() + 1)
    rew_coins = random.randint(1,100)
    rew_xp = random.randint(100,1000)
    player._progress.add_xp(rew_xp)
    player._progress.add_coins(rew_coins)
    Result.destroy()
    Battl.destroy()


def battle():
    hp = player.get_health()

    Battl = Toplevel(frame_game,bg='#878E99')
    Battl.title("Battle")
    Battl.geometry("350x500")

    if player.get_beaten_enemies()>len(Enemies)-1:
        lb = Label(Battl, text="You beat them all!")
        lb.pack(pady=5)
        btn = Button(Battl, text="OK", command=Battl.destroy)
        btn.pack(pady=5)
    else:
        enemy = Enemies[player.get_beaten_enemies()]
        e_hp = enemy.get_health()

        Hero_hp = Label(Battl, text=f"Your current hp {hp}", bg='#9DB5B2')
        Hero_damage = Label(Battl, text=f"Your current damage {player.get_damage()}", bg='#9DB5B2')
        Hero_heal = Label(Battl, text=f"Your current heal {player.get_heal_amount()}", bg='#9DB5B2')
        Enemy_hp = Label(Battl, text=f"Enemy's current hp {e_hp}", bg='#9DB5B2')
        Enemy_damage = Label(Battl, text=f"Enemy's damage {enemy.get_damage()}", bg='#9DB5B2')
        Enemy_name = Label(Battl, text=f"Enemy's name {enemy.get_name()}", bg='#9DB5B2')
        image_space = Enemies[player.get_beaten_enemies()].get_name()
        image_name = re.sub(r"\s","_",image_space)
        image_name = re.sub(r'"',"",image_name)
        image = Image.open(f"{image_name}.png")
        Battl.Enemy_img = ImageTk.PhotoImage(image)
        E_img = Label(Battl, image=Battl.Enemy_img, width=200, height=200)
        Batt = Battle(player,enemy,Battl)
        Dmg = Button(Battl,text="Deal damage", command=Batt.deal_damage, bg='#9DB5B2')
        Heal = Button(Battl,text="Heal", command=Batt.heal, bg='#9DB5B2')

        Dmg.place(x=50, y = 400)
        Heal.place(x=250, y = 400)
        E_img.place(x=70, y=120)
        Enemy_name.place(x=80,y=100)
        Enemy_damage.place(x=210,y=33)
        Enemy_hp.place(x=210,y=3)
        Hero_hp.place(x=3,y=3)
        Hero_damage.place(x=3,y=33)
        Hero_heal.place(x=3,y=63)
        Battl.after(500,lambda: upd_info(Hero_hp,Enemy_hp,Batt,Battl))

def upd_info(Hero_hp,Enemy_hp,Batt,Battl):
    Hero_hp.config(text=f"Your current hp {Batt.get_current_hp()}", bg='#9DB5B2')
    Enemy_hp.config(text=f"Enemy's current hp {Batt.get_enemy_current_hp()}", bg='#9DB5B2')
    Battl.after(500, lambda: upd_info(Hero_hp,Enemy_hp,Batt,Battl))

def claim_reward(enemy_hp,rew_coins,rew_xp,dBattle):
    if player._energy.get_daily_steps()>=enemy_hp and player.get_status()!= datetime.now().strftime("%d"):
        player._progress.add_coins(rew_coins)
        player._progress.add_xp(rew_xp)
        player.set_status(datetime.now().strftime("%d"))

    else:
        Error = Toplevel(dBattle,bg='#878E99')
        Error.title("Error")
        Er = Label(Error,text="First you need to kill boss! Or you have already took the reward!", bg='#9DB5B2')
        Er.pack(pady=5)
        
        exit = Button(Error,text="OK", command=Error.destroy, bg='#9DB5B2')
        exit.pack(pady=5)
async def timer():
    time.sleep(24*60*60)
    dBoss_killed = False

def daily_battle():
    dBattle = Toplevel(frame_game,bg='#878E99')
    dBattle.title("Daily Battle")
    dBattle.geometry("350x500")

    with open("DailyEnemies.txt", "r", encoding="utf-8") as file:
        enemies = [line.strip() for line in file if line.strip()]



    today = datetime.today().strftime("%Y-%m-%d")
    random.seed(today)
    enemy_number = random.randint(0, len(enemies)-1)
    enemy_hp = random.randint(2000, 7000)
    rew_xp = random.randint(50, 500)
    rew_coins = random.randint(5, 50)
    print(enemies[enemy_number])

    RewXP = Label(dBattle, text=f"Reward XP {rew_xp}", bg='#878E99')
    RewXP.place(x=10,y=10)
    RewCoins = Label(dBattle,text=f"Reward Coins {rew_coins}", bg='#878E99')
    RewCoins.place(x=250,y=10)

    Progr = Label(dBattle, text=f"{player._energy.get_daily_steps()}/{enemy_hp}", bg='#878E99')
    Progr.place(x=150,y=50)

    progress = ttk.Progressbar(dBattle, length=300, maximum=enemy_hp, )
    progress["value"] = player._energy.get_daily_steps()
    progress.place(x=25,y=100)

    done_button = Button(dBattle,text="Claim reward", command=lambda :claim_reward(enemy_hp,rew_coins,rew_xp,dBattle), bg='#9DB5B2')
    done_button.place(x=125,y=200)



def update_st(sleep_time,ST_window):
    player.set_sleep_time(sleep_time)
    ST_window.destroy()

def add_ST():
    ST_window = Toplevel(frame_game,bg='#878E99')
    ST_window.title("Add Sleep Time")

    ST_rn = Label(ST_window, text=f"You have slept {player.get_sleep_time()}", bg='#9DB5B2')
    ST_rn.pack()
    St_label = Label(ST_window, text="Enter how many hours you have slept", bg='#9DB5B2')
    St_label.pack(pady=5)
    St_enter = Entry(ST_window)
    St_enter.pack()
    save_st = Button(ST_window, text="OK", command=lambda: update_st(St_enter.get(), ST_window), bg='#9DB5B2')
    save_st.pack()


def update_hydration(hydration, hydration_window):
    player.set_hydration(hydration)
    hydration_window.destroy()

def add_hydration():


    hydration_window = Toplevel(frame_game,bg='#878E99')
    hydration_window.title("Add Hydration")

    Hydr_rn = Label(hydration_window, text=f"Today's hydration {player.get_hydration()}", bg='#9DB5B2')
    Hydr_rn.pack()
    Add_Hydr_Label = Label(hydration_window, text="How many liters of water have you drank today", bg='#9DB5B2')
    Add_Hydr_Label.pack(pady=5)
    entry_hydration = Entry(hydration_window)
    entry_hydration.pack()


    btn_confirm = Button(hydration_window,text="OK", command=lambda :update_hydration(entry_hydration.get(), hydration_window), bg='#9DB5B2')
    btn_confirm.pack(pady=5)

def update_info(hero_name, hero_hp,hero_damage,hero_heal,hero_level,hero_xp,hero_coins,all_time,steps,beaten_label):
    
    player._energy.update_all_time_steps()
    #player._energy.daily_steps_from_google()
    
    steps.config(text=f"Steps today: {str(player._energy.get_daily_steps())}")
    hero_level.config(text=f"Level {player._progress.get_level()}")
    hero_xp.config(text=f"XP {player._progress.get_xp()}")
    hero_coins.config(text=f"Coins {player._progress.get_coins()}")
    hero_damage.config(text=f"Damage {player.get_damage()}")
    hero_name.config(text=f"Hero {player.get_name()}")
    hero_hp.config(text=f"Hp {player.get_health()}")
    hero_heal.config(text=f"Heal {player.get_heal_amount()}")
    all_time.config(text=f"All Time Steps {player._energy.get_all_time_steps()}")
    beaten_label.config(text=f"Beaten enemies {player.get_beaten_enemies()}")
    player.save_to_db()

    frame_game.after(2000, lambda: update_info(hero_name, hero_hp, hero_damage, hero_heal, hero_level, hero_xp, hero_coins, all_time, steps, beaten_label))

def start_game(name_t,age_t,height_t,weight_t,classe_t,gender_t,xp,level,coins,daily_steps,all_time_s,beaten_enemies,emaill,status,frame_start):

    
    global player, steps, hero_level
    print (f"Email {emaill}")
    emaill = email

    username = name_t
    age = age_t
    height = height_t
    weight = weight_t
    classe = classe_t
    gender = gender_t
    

    if not username:
        return

    if classe == "Warrior":
        player = Warrior(username, age, gender, height, weight,email)
    else:
        player = Mage(username, age, gender, height, weight,email)
        
    print(xp)
    print(f"COINS {coins}")
    player._progress.set_xp(xp)
    
    print(f"COINNSSSSSSSS {player._progress.get_coins()}")
    print(f"XPPPPPPPPPPPPPP {player._progress.get_xp()}")

    player.set_status(status)
    player._progress.set_coins(coins)
    player._energy.set_all_time_steps(all_time_s)
    player._energy.set_daily_steps(daily_steps)
    player._energy.update_all_time_steps()
    player.set_beaten_enemies(beaten_enemies)


    player.set_email(emaill)


    if frame_start:
        
        frame_start.pack_forget()

    
    
    frame_game.pack(fill="both", expand=True)
   

    hero_name = Label(frame_game, text=f"Hero: {player.get_name()}", bg="#878E99", fg="white", font=("Arial", 12, "bold"))
    hero_name.place(x=20, y=20)

    hero_hp = Label(frame_game,text=f"Hp {player.get_health()}", bg="#878E99", fg="white", font=("Arial", 12, "bold"))
    hero_hp.place(x=20, y=50)

    hero_damage= Label(frame_game,text=f"Damage {player.get_damage()}", bg="#878E99", fg="white", font=("Arial", 12, "bold"))
    hero_damage.place(x=20, y=80)

    hero_heal= Label(frame_game,text=f"Heal {player.get_heal_amount()}", bg="#878E99", fg="white", font=("Arial", 12, "bold"))
    hero_heal.place(x=20, y=110)

    hero_level = Label(frame_game, text=f"Level {player._progress.get_level()}", bg="#878E99", fg="white", font=("Arial", 10))
    hero_level.place(x=250, y=20)

    hero_xp = Label(frame_game, text=f"XP {player._progress.get_xp()}", bg="#878E99", fg="white", font=("Arial", 10))
    hero_xp.place(x=250, y=50)

    hero_coins = Label(frame_game, text=f"Coins {player._progress.get_coins()}", bg="#878E99", fg="white", font=("Arial", 10))
    hero_coins.place(x=250, y=80)
    if player._energy.get_daily_steps()>player._energy.get_all_time_steps():
        x= player._energy.get_daily_steps()
    else:
        x = player._energy.get_all_time_steps()
    all_time = Label(frame_game, text=f"All time steps {x}", bg="#878E99", fg="white", font=("Arial", 10))
    all_time.place(x=220, y=110)

    steps = Label(frame_game, text=f"Steps today {player._energy.get_daily_steps()}", bg="#878E99", fg="white", font=("Arial", 18))
    steps.place(x=70, y=140)
    day_quote = Label(frame_game, text=f"Daily quote\n{quotes[quotes_number]}", font=("Arial",15), bg="#878E99", fg="white",wraplength=200)
    day_quote.place(x=70, y=210)

    btn_add_hydration = Button(frame_game, text="Add Hydration", command=add_hydration, width=7, height=5, wraplength=30)
    btn_add_hydration.place(x=3, y=140)

    btn_add_sleep_time = Button(frame_game,text="Add Sleep Time", command=add_ST, width=7,height=5,wraplength=30)
    btn_add_sleep_time.place(x=3, y=230)

    btn_daily_battle = Button(frame_game, text="Daily Battle", command=daily_battle,width=7,height=5,wraplength=40)
    btn_daily_battle.place(x=290, y=140)

    btn_battle = Button(frame_game,text="Battle", command=battle, width=20, height=3)
    btn_battle.place(x=100, y=350)

    btn_change_name = Button(frame_game,text="Change name", command=changing, width=7, height=5, wraplength=30)
    btn_change_name.place(x=290, y=230)

    beaten_label = Label(text=f"Beaten enemies {player.get_beaten_enemies()}", bg="#878E99", fg="white")
    beaten_label.place(x=120,y=410)
    update_info(hero_name, hero_hp,hero_damage,hero_heal,hero_level,hero_xp,hero_coins,all_time,steps,beaten_label)

    # ======= Start Frame =======
def start():

    frame_start = Frame(root, padx=20, pady=20, bg="#878E99")
    frame_start.pack()


    label = Label(frame_start, text="Enter your hero's name:", bg='#9DB5B2')
    label.pack(pady=5)
    entry_name = Entry(frame_start, bg='#9DB5B2')
    entry_name.pack()


    label_age = Label(frame_start, text="Enter your age:", bg='#9DB5B2')
    label_age.pack(pady=5)
    entry_age = Entry(frame_start)
    entry_age.pack()


    label_height = Label(frame_start, text="Enter your height (m):", bg='#9DB5B2')
    label_height.pack(pady=5)
    entry_height = Entry(frame_start)
    entry_height.pack()


    label_weight = Label(frame_start, text="Enter your weight (kg):", bg='#9DB5B2')
    label_weight.pack(pady=5)
    entry_weight = Entry(frame_start)
    entry_weight.pack()


    label_gender = Label(frame_start, text="Select your gender:", bg='#9DB5B2')
    label_gender.pack(pady=5)
    gender_options = ["Male", "Female"]
    gender_var = StringVar()
    gender_var.set(gender_options[0])
    gender_menu = OptionMenu(frame_start, gender_var, *gender_options)
    gender_menu.pack()


    label_class = Label(frame_start, text="Select your class:", bg='#9DB5B2')
    label_class.pack(pady=5)
    class_options = ["Warrior", "Mage"]
    class_var = StringVar()
    class_var.set(class_options[0])
    class_menu = OptionMenu(frame_start, class_var, *class_options)
    class_menu.pack()
    button = Button(frame_start, text="Create Character", command=lambda: start_game(entry_name.get(),entry_age.get(),entry_height.get(),entry_weight.get(),class_var.get(),gender_var.get(),0,0,0,0,0,0,0,0,frame_start))
    button.pack(pady=10)

def check_is():
    global email
    email = "test@test.com"

    with sqlite3.connect("game.db", timeout=10, check_same_thread=False) as conn:
        cursor = conn.cursor()

        
        cursor.execute("""SELECT * from characters where email = ?""", (email,))
        row = cursor.fetchone()
        print(email)
        
    if row is None:
        print("123")
        start()
    else:
        with sqlite3.connect("game.db", timeout=10, check_same_thread=False) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM characters WHERE email = ?", (email,))
            row = cursor.fetchone()
            
            cursor.close()
            print(row)
            print("\n")
            print(row[0])
            id = row[0] if row[0] is not None else 0
            name = row[1] if row[1] is not None else ""
            age = row[2] if row[2] is not None else 0
            gender = row[3] if row[3] is  None else ""
            weight = row[4] if row[4] is   None else 0
            height = row[5] if row[5] is  None else 0
            xp = row[6] if row[6] is not None else 0
            level = row[7] if row[7] is  None else 0
            coins = row[8] if row[8] is not None else 0
            daily_steps = row[9] if row[9] is not None else 0
            all_time_s = row[10] if row[10] is not None else 0
            beaten_enemies = row[11] if row[11] is not None else 0
            emaill = row[12] if row[12] is  None else ""
            classi = row[13] if row[13] is  None else ""
            status = row[14] if row[14] is None else 0
            name = "Test"
            age = 19
            gender = "Male"
            weight = 64.2
            height = 1.8
            xp = 306
            coins = 6000
            daily_steps = 1000
            all_time_s = 20000
            beaten_enemies = 0
            emaill = "test@test.com"
            classi = "Warrior"
            status = 1
            for i in range(0,13):
                print(row[i])
                
                
            start_game(name, age, height, weight, classi, gender, xp, level, coins, daily_steps, all_time_s, beaten_enemies, emaill,status,frame_start=None)



check_is()

root.mainloop()


6
test@test.com
123
Email 0
0
COINS 0
COINNSSSSSSSS 0
XPPPPPPPPPPPPPP 0
Saving...
Saving...
Saving...
Saving...
Saving...
Saving...
Saving...
Saving...


In [11]:
with sqlite3.connect("game.db", timeout=10, check_same_thread=False) as conn:
    cursor = conn.cursor()


    cursor.execute("SELECT * FROM characters")


    rows = cursor.fetchall()


    if rows:
        for row in rows:
            print(row) 
    else:
        print("No data")

('ad74099b-8dc7-45bc-bd3c-f0f3d474fdd1', '12312', 213, 'Male', 11.0, 123.0, 674, 1, 28, 0, 0, 1, 'test@test.com', 'Warrior')


In [45]:
datetime.now().strftime("%d")

'28'